In [ ]:
from torchvision.transforms import ToTensor
import rasterio
from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
from glob import glob
from typing import Tuple, Dict
import json
import geopandas as gpd

In [ ]:
# image = tiff.imread('/kaggle/input/solafune/train_images/train_images/train_1.tif')
def plot_image(path, gdf):
    with rasterio.open(path) as image:
        image_array = image.read()

    print(image_array.shape)
    image_array = image_array.transpose(0, 2, 1)
        
    band_red = image_array[3, :, :]  
    band_green = image_array[2, :, :] 
    band_blue = image_array[1, :, :]  
    band_red = np.nan_to_num(band_red, nan=0)
    band_green = np.nan_to_num(band_green, nan=0)
    band_blue = np.nan_to_num(band_blue, nan=0)
    
    rgb_image = np.dstack((band_red, band_green, band_blue))
    image = (rgb_image - np.min(rgb_image)) / (np.max(rgb_image) - np.min(rgb_image))
    image = np.flipud(image)
    # Convert to uint8
    image = (image * 255).astype(np.uint8)
    fig, (ax) = plt.subplots(1,1, figsize=(10, 10))
    ax.imshow(image)
    # show_anns(masks)
    # ax.axis('off')
    # gdf.plot(ax=ax, column='class',legend=True,figsize=(20, 20))
    gdf.boundary.plot(ax=ax, figsize=(10, 10))
    plt.show() 

In [ ]:
with open("./data/train_annotations.json") as f:
    data = json.load(f)

json_data = data["images"][0]["annotations"]
path = "./data/train_images/train_0.tif"


def convert_to_geojson(data):
  """
  Converts a list of dictionaries in the specified format to GeoJSON

  Args:
      data: A list of dictionaries containing 'class' and 'segmentation' keys

  Returns:
      A GeoJSON feature collection
  """
  features = []
  for item in data:
    polygon = []
    for i in range(0, len(item['segmentation']), 2):
      polygon.append([item['segmentation'][i], item['segmentation'][i+1]])
    features.append({
      "type": "Feature",
      "geometry": {
        "type": "Polygon",
        "coordinates": [polygon]
      },
      "properties": {"class": item['class']}
    })
  return { "type": "FeatureCollection", "features": features}


# Convert to GeoJSON
geojson_data = convert_to_geojson(json_data)
gdf = gpd.GeoDataFrame.from_features(geojson_data)

In [ ]:
plot_image(path, gdf)